In [7]:
import pandas as pd
from pathlib import Path

In [8]:
# Original Excel file
national_level = "/Users/mariajosereyesramirez/Documents/CAPP/2ND YEAR/Autumn quarter 2025/Data Viz/interactive_project/interactive_code/raw_data/1.2_Prevalence_of_drug_use_in_the_general_population_national_data (1).xlsx"
# iso codes
iso_codes = "/Users/mariajosereyesramirez/Documents/CAPP/2ND YEAR/Autumn quarter 2025/Data Viz/interactive_project/final_version/UNCountries.csv"
# Folder to store Json files
output_dir = Path("/Users/mariajosereyesramirez/Documents/CAPP/2ND YEAR/Autumn quarter 2025/Data Viz/interactive_project/final_version/drugs_directory")

In [9]:
def load_country_codes():
    """
    Load UN country codes CSV and prepare it for merging.
    """
    # separating col names
    codes = pd.read_csv(iso_codes, sep="\t")
    codes.columns = codes.columns.str.strip()

    print("Columns in UNCountries.csv:", list(codes.columns))

    # Defining column names
    country_col_un = "Country or Area"
    iso3_col_un = "ISO-alpha3 code"
    m49_col_un = "M49 code"

    # Normalizing country names for merging
    codes["country_key"] = (
        codes[country_col_un]
        .astype(str)
        .str.strip()
        .str.upper()
    )

    codes = codes[["country_key", iso3_col_un, m49_col_un]].rename(
        columns={
            iso3_col_un: "iso3",
            m49_col_un: "m49",
        }
    )

    return codes


In [10]:
codes = pd.read_csv(iso_codes)
codes.columns = codes.columns.str.strip()
print(list(codes.columns))

['Country or Area\tM49 code\tISO-alpha3 code']


In [11]:
def main():
    output_dir.mkdir(exist_ok=True)
    #Loading country codes
    country_codes = load_country_codes()

    consumption_data = pd.ExcelFile(national_level, engine="openpyxl")
    print("Sheets found:")
    print(consumption_data.sheet_names)

    # Columns I want to keep
    cols_to_keep = ["Country/Territory", "Best", "Male", "Female", "Year"]

    for sheet_name in consumption_data.sheet_names:
        print(f"\nProcessing sheet: {sheet_name}")
        # Reading each sheet
        national_consumption = pd.read_excel(consumption_data, sheet_name=sheet_name)
        # Cleaning column names
        national_consumption.columns = national_consumption.columns.str.strip()

        # Checking that all needed cols exist
        missing = [c for c in cols_to_keep if c not in national_consumption.columns]
        if missing:
            print(f" Skipping{sheet_name}(missing columns:{missing})")
            continue

        # Keeping only the cols I need (Country, Year, Best, Male, Female)
        national_consumption = national_consumption[cols_to_keep].copy()

        # Creating merging key
        national_consumption["country_key"] = (
            national_consumption["Country/Territory"]
            .astype(str)
            .str.strip()
            .str.upper()
        )

        # Merging with UN country codes
        national_consumption = national_consumption.merge(
            country_codes,
            on="country_key",
            how="left"
        )

        # Warning about unmatched countries
        unmatched = national_consumption[national_consumption["iso3"].isna()]["Country/Territory"].unique()
        if len(unmatched) > 0:
            print(f"  Warning: {len(unmatched)} countries without ISO/M49 match (showing up to 10):")
            print("   ", unmatched[:10])

        # Dropping helper column
        national_consumption = national_consumption.drop(columns=["country_key"])

        # Confirming numeric values for year
        national_consumption["Year"] = pd.to_numeric(
            national_consumption["Year"],
            errors="coerce"
        )
        national_consumption = national_consumption.dropna(subset=["Year"])

        # Filtering for years from 2019
        national_consumption = national_consumption[national_consumption["Year"] >= 2019]
        national_consumption["Year"] = national_consumption["Year"].astype(int)

        if national_consumption.empty:
            print(f"  No rows from 2019 onwards in {sheet_name}, skipping JSON.")
            continue
        
        national_consumption["Best"] = pd.to_numeric(
            national_consumption["Best"],
            errors='coerce'
        )

        def add_country_stats(group):
            group = group.sort_values("Year")
            first_year = int(group["Year"].iloc[0])
            last_year = int(group["Year"].iloc[-1])

            first_best = group["Best"].iloc[0]
            last_best =group["Best"].iloc[-1]

            if pd.notna(first_best) and pd.notna(last_best):
                diff = float(last_best - first_best)
            else:
                diff = None

            group["max_year"] = last_year
            group["difference"] = diff
            return group
        
        national_consumption = (
            national_consumption
            .groupby("Country/Territory", group_keys=False)
            .apply(add_country_stats)
        )


        # Reordering columns so ISO/M49 are next to country
        cols_order = [
            "Country/Territory",
            "iso3",
            "m49",
            "Best",
            "Male",
            "Female",
            "Year",
            "max_year",
            "difference",
        ]
        cols_order = [c for c in cols_order if c in national_consumption.columns]
        national_consumption = national_consumption[cols_order]

        # Building JSON file name for sheet name (drug name)
        safe_name = sheet_name.strip().lower().replace(" ", "_")
        json_path = output_dir / f"{safe_name}.json"

        national_consumption.to_json(json_path, orient="records", indent=2)
        print(f"  Saved: {json_path}")


if __name__ == "__main__":
    main()

Columns in UNCountries.csv: ['Country or Area', 'M49 code', 'ISO-alpha3 code']
Sheets found:
['Cannabis', 'Cocaine', 'Amphetamines', 'Ecstasy', 'Prescription Stimulants', 'Opioids', 'Opiates', 'Prescription opioids', 'Tranquillizers and sedatives', 'NPS']

Processing sheet: Cannabis
    ["Côte d'Ivoire" 'China, Macao SAR' 'China, Taiwan Province of China'
 'Kosovo under UNSCR 1244' 'United Kingdom (England and Wales)'
 'United Kingdom (Northern Ireland)' 'United Kingdom (Scotland)' nan]
  Saved: /Users/mariajosereyesramirez/Documents/CAPP/2ND YEAR/Autumn quarter 2025/Data Viz/interactive_project/final_version/drugs_directory/cannabis.json

Processing sheet: Cocaine


/var/folders/lj/c5xx9t6915gdhp5gk9xqmv2h0000gn/T/ipykernel_1044/2124963938.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  national_consumption
/var/folders/lj/c5xx9t6915gdhp5gk9xqmv2h0000gn/T/ipykernel_1044/2124963938.py:91: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  national_consumption
/var/folders/lj/c5xx9t6915gdhp5gk9xqmv2h0000gn/T/ipykernel_1044/2124963938.py:91: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns

    ["Côte d'Ivoire" 'China, Taiwan Province of China'
 'United Kingdom (England and Wales)' 'United Kingdom (Northern Ireland)'
 'United Kingdom (Scotland)' nan]
  Saved: /Users/mariajosereyesramirez/Documents/CAPP/2ND YEAR/Autumn quarter 2025/Data Viz/interactive_project/final_version/drugs_directory/cocaine.json

Processing sheet: Amphetamines
    ["Côte d'Ivoire" 'China, Taiwan Province of China'
 'United Kingdom (England and Wales)' 'United Kingdom (Northern Ireland)'
 'United Kingdom (Scotland)' nan]
  Saved: /Users/mariajosereyesramirez/Documents/CAPP/2ND YEAR/Autumn quarter 2025/Data Viz/interactive_project/final_version/drugs_directory/amphetamines.json

Processing sheet: Ecstasy
    ['China, Taiwan Province of China' 'United Kingdom (England and Wales)'
 'United Kingdom (Northern Ireland)' 'United Kingdom (Scotland)' nan]
  Saved: /Users/mariajosereyesramirez/Documents/CAPP/2ND YEAR/Autumn quarter 2025/Data Viz/interactive_project/final_version/drugs_directory/ecstasy.json

P

/var/folders/lj/c5xx9t6915gdhp5gk9xqmv2h0000gn/T/ipykernel_1044/2124963938.py:91: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  national_consumption
/var/folders/lj/c5xx9t6915gdhp5gk9xqmv2h0000gn/T/ipykernel_1044/2124963938.py:91: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  national_consumption
